In [1]:
import pyc3dserver as c3d
itf = c3d.c3dserver(False)

import pandas as pd
import math
import numpy as np

def getData(file_name, number=0, close_other=True):#les données intéressantes du fichier c3d
    if close_other:#sait pas si ça marche
        ret=c3d.close_c3d(itf)
    ret = c3d.open_c3d(itf, file_name)

    n_frames = c3d.get_last_frame(itf)
    lab=c3d.get_marker_names(itf)
    data=c3d.get_dict_markers(itf)['DATA']['POS']
    
    ret=c3d.close_c3d(itf)
    
    #Si toutes les noms des trajectoires commencent avec les memes caractères (ex: New), on supprime ces caractères redondants
    for i in range(1, len(lab[0])):
        if all(list(t[0:i]==lab[0][0:i] for t in lab)):
            pass;
        else:
            i-=1
            lab=[e[i:] for e in lab]
            datak=[e[i:] for e in data.keys()]
            data=dict(zip(datak, list(data.values())))
            break;
    
    labdico = {}
    for i in lab:
        df = pd.DataFrame(data[i], columns=['x', 'y', 'z']).apply(pd.to_numeric)
        df = df[(df.x!=0) & (df.y!=0) & (df.z!=0)]
        if not df.empty:
            labdico[i]=df.copy()       

    return n_frames, list(labdico.keys()), labdico


#vrai si le marqueur lab commence à la frame
def is_new(lab, frame):
    return labdico[lab].index[0]==frame

#vrai si le marqueur lab fini à la frame
def is_old(lab, frame):
    return labdico[lab].index[-1]==frame

#vrai si le marqueur lab existe à la frame
def is_present(lab, frame):
    return frame in lab.index

#liste les marqueurs présents à la frame
def markers_present(frame):
    mark=[]
    for i in labdico:
        if is_present(labdico[i], frame):
            mark.append(i)
    return mark
    
#retourne le nombre de marqueur présents à la frame
def n_markers_present(frame):
    n=0
    for i in labdico:
        if is_present(labdico[i], frame):
            n+=1
    return n
    
#vrai si 2 marqueurs (représentés par des dataframes) se superposent
def superpos(df1, df2):
    if (df1.index[-1]<df2.index[0]) or (df2.index[-1]<df1.index[0]):
        return False
    else: return True
    

def avg_speed(traj, step):
    if len(labdico[traj])>step:
        cpt=0
        speed=0
        for frame in range(labdico[traj].index[0], labdico[traj].index[-1]-step, step):
            pos_future=labdico[traj].loc[frame+step]
            speed+=math.dist(labdico[traj].loc[frame], pos_future)
            cpt+=1
        return speed/(cpt*step)

def get_key_from_value(dico, val):
    keys = [k for k, v in dico.items() if val in v]
    if keys:
        return keys[0]
    return None

#liste des tetes et queue du dictionaire connect
def get_head_tail_connect():
    if len(connect)==0:
        return [], []
    else:
        head=[]
        tail=[]
        for key, val in connect.items():
            head = [*head, val[0]]
            tail = [*tail, val[-1]]
        return head, tail

connect ={}
def connexion(lab1, lab2):#enrichie le dictionnaire connect en connectant les marqueurs à la chaine
    head, tail = get_head_tail_connect()
    if tail==None:
        connect[lab1] = [lab1, lab2]
    else:
        if (lab1 in tail) and (lab2 in head):
            key1=get_key_from_value(connect, lab1)
            connect[key1]= connect[key1]+connect[lab2]
            del connect[lab2]
        elif lab1 in tail:
            key=get_key_from_value(connect, lab1)
            connect[key].insert(len(connect[key]), lab2)
        elif lab2 in head:
            connect[lab2].insert(0, lab1)
            connect[lab1] = connect.pop(lab2)
        else :
            connect[lab1] = [lab1, lab2]

#===============================================================================================
#===============================================================================================
# Fonctions pour supprimer les artefacts:

def find_artifacts1(labdico, step, dist_max):#Supprime les marqueurs loin des autres en regardant toute les "artif_detec_step1" frames
    
    artef=[]
    frames_seen=[]
    
    for frame in range(step ,n_frames-step, step):
        mark_pres=markers_present(frame)
        #On dégage les marqueurs distants:
        for i in mark_pres:
            distant=True
            for j in mark_pres:
                if i!=j:
                    if math.dist(labdico[i].loc[frame], labdico[j].loc[frame]) < dist_max:
                        distant=False
                        break
            if distant:
                artef.append(i)
           
    return sorted(list(dict.fromkeys(artef)))

#===============================================================================================
def find_artifacts2(labdico, dist_max):#Supprime les marqueurs si loins des autres a leur naissance et mort
    
    artef=[]
    frames_seen=[]
    
    for r in [0, -1]:#0 pour la première frame, -1 pour la dernière
        frames_seen=[]
        for h in labdico:
            if not labdico[h].index[r] in frames_seen: #si n'a pas été vu:
                frames_seen.append(labdico[h].index[r]) #on l'ajoute à la liste, frame_seen[-1] devient la frame intéressante
                mark_pres = markers_present(frames_seen[-1]) #les mark qui sont là dans la frame
                mark=[]


                for i in mark_pres:
                    if r==0 and is_new(i, frames_seen[-1]):
                        mark.append(i) #les mark qui apparaissent avec la frame
                    elif r==-1 and is_old(i, frames_seen[-1]):
                        mark.append(i) #les mark qui disparaissent avec la frame

                #Les marqueurs qui apparaissent ou disparaissent sont ils loins des autres marqueurs présents à la frame d'apparition/de disparition ?
                for i in mark:
                    distant=True
                    for j in mark_pres:
                        if i!=j:
                            if math.dist(labdico[i].iloc[r], labdico[j].loc[frames_seen[-1]]) < dist_max:
                                distant=False
                                break
                    if distant:
                        artef.append(i)

    return sorted(list(dict.fromkeys(artef)))
    
#===============================================================================================
def find_artifacts3(labdico, step, dist_motion):#Supprime les marqueurs si ne bougent pas assez (si vitesse maximum inférieur à dist_motion)
    
    artef=[]
    
    for i in labdico:
        if len(labdico[i])>step:
            motion=False
            for frame in range(labdico[i].index[0], labdico[i].index[-1]-step, step):
                pos_future=labdico[i].loc[frame+step]
                if math.dist(labdico[i].loc[frame], pos_future) > dist_motion:
                    motion=True
                    break
            if not motion:
                artef.append(i)
                
    return sorted(list(dict.fromkeys(artef)))

#===============================================================================================
def find_artifacts4(labdico, step, dist_motion):#Supprime les marqueurs si ne bougent pas assez (si vitesse moyenne inférieur à dist_motion)
    
    artef=[]
    
    for i in labdico:
        if len(labdico[i])>step:
            if avg_speed(i, step)<dist_motion:
                artef.append(i)
                
    return sorted(list(dict.fromkeys(artef)))

#===============================================================================================
def find_artifacts5(labdico, n_marqueurs):
    
    end_dict={} #une clé de end_dict est le nom d'une trajectoire, sa valeur est l'entier correspondant à sa dernière frame
    for i in labdico:
        end_dict[i]=labdico[i].index[-1]
    end_dict = dict(sorted(end_dict.items(), key=lambda item: item[1]))# les trajectoires qui se terminent en premier sont les premieres du dictionnaire 

    proba_artef={}
    end_fr=0
    for i in labdico:
        frame_start=labdico[i].index[0]# on regarde à l'apparition de chaque trajectoire si on a plus de 15 marqueurs
        if n_markers_present(frame_start)>n_marqueurs and frame_start>end_fr: #et après end_fr pour pas refaire les memes choses
            for j in end_dict:# on cherche j: le marqueur qui a sa disparition fait qu'il n'y a plus plus de 15 marqueurs
                if end_dict[j]>frame_start and n_markers_present(end_dict[j])<=15:
                    for k in markers_present(frame_start): #on ajoute chaque marqueurs présents à proba_artef
                        if not k in proba_artef:
                            proba_artef[k]=0

                        if len(labdico[k])>200:
                            step=100
                        else:
                            step=10

                        avg = avg_speed(k, step)
                        #if avg==avg:
                        if isinstance(avg, int):
                            print(avg)
                            proba_artef[k]+=avg_speed(k, step)

                    end_fr=end_dict[j]
                    break;#quitte la boucle de end_dict car on vient de trouver le end


    if proba_artef: #si le dico n'est pas vide
        return list({k: v for k, v in sorted(proba_artef.items(), key=lambda item: item[1])}.keys())[0]
    else:
        return []
    
#==================================================================================================
def delete_artifacts(lab, labdico):
    
    artifacts = list(dict.fromkeys(find_artifacts3(labdico, 75, 3.5)+find_artifacts4(labdico, 80, 0.01625)))
    lab=[x for x in lab if x not in set(artifacts)]
    for i in artifacts:
        del labdico[i]

    artif = artifacts
    artifacts = []
    
    while True:
        artifacts=list(dict.fromkeys(find_artifacts1(labdico, 90, 670)+find_artifacts2(labdico, 625)))
        artif += artifacts
        if artifacts == []:
            break;
        lab=[x for x in lab if x not in set(artifacts)]
        for i in artifacts:
            del labdico[i]

    while True:
        artifacts=find_artifacts5(labdico, 15)
        if artifacts == []:
            break;
        else:
            lab=[x for x in lab if x not in set(artifacts)]
            del labdico[artifacts]#artifact a une taille de 1, donc pas besoin de faire de boucle
            artif += [artifacts]

    return sorted(artif)

In [2]:
#connectl est le dictionnaire qui correspond aux vrais connexions

connectl03={'tete': ['New 0013', 'New 0052', 'New 0059', 'New 0073', 'New 0075', 'New 0082', 'New 0087', 'New 0093', 'New 0100', 'New 0104', 'New 0107', 'New 0116', 'New 0126', 'New 0133', 'New 0156', 'New 0164', 'New 0177'],
'torse': ['New 0000', 'New 0020', 'New 0026', 'New 0040', 'New 0043', 'New 0048', 'New 0051', 'New 0054', 'New 0056', 'New 0072', 'New 0085', 'New 0089', 'New 0098', 'New 0106', 'New 0125', 'New 0139', 'New 0141', 'New 0155', 'New 0163', 'New 0176'],
'G_epaule': ['New 0008', 'New 0042', 'New 0046', 'New 0086', 'New 0157', 'New 0172'],
'D_epaule': ['New 0003', 'New 0153'],
'G_coude': ['New 0006', 'New 0045', 'New 0109', 'New 0121', 'New 0143', 'New 0173'],
'G_main': ['New 0002', 'New 0033', 'New 0064', 'New 0090', 'New 0144', 'New 0171'],
'G_hanche': ['New 0014', 'New 0015', 'New 0016', 'New 0021', 'New 0038', 'New 0044', 'New 0050', 'New 0060', 'New 0110', 'New 0118', 'New 0122', 'New 0145', 'New 0175'],
'G_genou': ['New 0011', 'New 0023', 'New 0025', 'New 0091', 'New 0102', 'New 0129', 'New 0136', 'New 0151', 'New 0174'],
'G_pied': ['New 0010', 'New 0029', 'New 0035', 'New 0055', 'New 0111', 'New 0130', 'New 0134', 'New 0137', 'New 0140', 'New 0147', 'New 0168', 'New 0180'],
'D_genou': ['New 0007', 'New 0092', 'New 0099', 'New 0148', 'New 0159', 'New 0161'],
'D_pied': ['New 0012', 'New 0024', 'New 0030', 'New 0031', 'New 0036', 'New 0063', 'New 0069', 'New 0071', 'New 0080', 'New 0088', 'New 0096', 'New 0108', 'New 0113', 'New 0120', 'New 0123', 'New 0131', 'New 0138', 'New 0146', 'New 0152', 'New 0166', 'New 0170', 'New 0179'],
'D_coude': ['New 0001', 'New 0027', 'New 0095', 'New 0124'],
'D_main': ['New 0009', 'New 0017', 'New 0028', 'New 0039', 'New 0053', 'New 0097', 'New 0115', 'New 0149', 'New 0160'],
'bassin': ['New 0005', 'New 0018', 'New 0049', 'New 0058', 'New 0061', 'New 0062', 'New 0066', 'New 0067', 'New 0074', 'New 0076', 'New 0103', 'New 0114', 'New 0128', 'New 0135', 'New 0158', 'New 0167', 'New 0178'],
'D_hanche': ['New 0004', 'New 0019', 'New 0022', 'New 0032', 'New 0034', 'New 0037', 'New 0047', 'New 0057', 'New 0065', 'New 0068', 'New 0077', 'New 0078', 'New 0081', 'New 0083', 'New 0084', 'New 0094', 'New 0101', 'New 0105', 'New 0112', 'New 0117', 'New 0119', 'New 0127', 'New 0132', 'New 0142', 'New 0150', 'New 0154', 'New 0162', 'New 0165', 'New 0169']}


connectl04={'tete': ['New 0005', 'New 0016', 'New 0049'],
'torse': ['New 0001',   'New 0018',   'New 0025',   'New 0031',   'New 0034',   'New 0036',   'New 0041',   'New 0052',   'New 0055',   'New 0058',   'New 0063',   'New 0069',   'New 0074',   'New 0093',   'New 0107',   'New 0123',   'New 0132',   'New 0144',   'New 0158'], 
'G_epaule': ['New 0003', 'New 0141'], 
'D_epaule': ['New 0000', 'New 0050', 'New 0139'], 
'G_coude': ['New 0006', 'New 0060', 'New 0138'],  
'G_main': ['New 0013',   'New 0022',   'New 0029',   'New 0045',   'New 0083',   'New 0118',   'New 0146'], 
'G_hanche': ['New 0008',   'New 0020',   'New 0027',   'New 0044',   'New 0048',   'New 0053',   'New 0065',   'New 0082',   'New 0098',   'New 0102',   'New 0119',   'New 0143'], 
'G_genou': ['New 0009',   'New 0021',   'New 0030',   'New 0057',   'New 0064',   'New 0086',   'New 0094',   'New 0117',   'New 0128',   'New 0140'], 
'G_pied': ['New 0004',   'New 0024',   'New 0033',   'New 0046',   'New 0056',   'New 0071',   'New 0087',   'New 0111',   'New 0120',   'New 0124',   'New 0142',   'New 0148',   'New 0154',   'New 0161'],  
'D_genou': ['New 0002',   'New 0039',   'New 0089',   'New 0099',   'New 0150',   'New 0156',   'New 0162',   'New 0165'], 
'D_pied': ['New 0012',  'New 0019',  'New 0032',   'New 0043',   'New 0078',   'New 0085',   'New 0090',   'New 0092',   'New 0100',   'New 0103',   'New 0108', 'New 0112',  'New 0114',   'New 0121',   'New 0133',   'New 0151',   'New 0155',   'New 0160'], 
'D_hanche': ['New 0014',   'New 0035',   'New 0042',   'New 0051',   'New 0059',   'New 0073',   'New 0077',   'New 0088',   'New 0135',   'New 0153',   'New 0164'], 
'D_coude': ['New 0010', 'New 0075', 'New 0138'], 
'D_main': ['New 0011',   'New 0017',   'New 0026',   'New 0038',   'New 0070',   'New 0072',   'New 0076',   'New 0081',   'New 0095',   'New 0146'],
'bassin': ['New 0007',   'New 0023',   'New 0028',   'New 0040',   'New 0047',   'New 0054',   'New 0061',   'New 0067',   'New 0079',   'New 0080',   'New 0084',   'New 0096',   'New 0101',   'New 0127',   'New 0149',   'New 0152',   'New 0163']} 

connectl05={'tete': ['New 0013', 'New 0016', 'New 0028', 'New 0041', 'New 0082'],
'torse': ['New 0000',   'New 0038',   'New 0042',   'New 0050',   'New 0053',   'New 0056',   'New 0071',   'New 0079',   'New 0114',   'New 0119',   'New 0129',   'New 0137'],
'G_epaule': ['New 0010', 'New 0017', 'New 0128'],
'D_epaule': ['New 0008', 'New 0031', 'New 0043'],
'G_coude': ['New 0004',   'New 0052',   'New 0101',   'New 0105',   'New 0127',   'New 0130',   'New 0136'],
'G_main': ['New 0005',   'New 0018',   'New 0024',   'New 0047',   'New 0061',   'New 0108',   'New 0126',   'New 0135'],
'G_hanche': ['New 0011',   'New 0025',   'New 0034',   'New 0070',   'New 0074',   'New 0081',   'New 0124'], 
'G_genou': ['New 0006',   'New 0021',   'New 0026',   'New 0066',   'New 0069',   'New 0125',   'New 0131'], 
'G_pied': ['New 0007',   'New 0020',  'New 0022',   'New 0033',   'New 0060', 'New 0064', 'New 0067',   'New 0072', 'New 0076',   'New 0088',   'New 0109',   'New 0112',   'New 0120',   'New 0123'], 
'D_genou': ['New 0003',   'New 0023',   'New 0030',   'New 0037',   'New 0049',   'New 0058',   'New 0063',   'New 0078',   'New 0084',   'New 0093',   'New 0098',   'New 0133'], 
'D_pied': ['New 0002',   'New 0029',   'New 0039',   'New 0057',   'New 0068',   'New 0077',   'New 0083',   'New 0087',   'New 0094',   'New 0104',   'New 0110',   'New 0113',   'New 0117',   'New 0122',   'New 0134'], 
'D_hanche': ['New 0001',   'New 0065',   'New 0075',   'New 0085',   'New 0095',   'New 0100',   'New 0115',   'New 0118',   'New 0132'],
'D_coude': ['New 0009', 'New 0027', 'New 0092'], 
'D_main': ['New 0012',   'New 0019',   'New 0032',   'New 0036',   'New 0044',   'New 0048',   'New 0086',   'New 0096',   'New 0099',   'New 0106'], 
'bassin': ['New 0015',   'New 0046',   'New 0059',   'New 0062',   'New 0073',   'New 0103',   'New 0107',   'New 0111',   'New 0116',   'New 0121',   'New 0138']} 

connectl06={'tete': ['New 0007',   'New 0016',   'New 0023',   'New 0028',   'New 0035',   'New 0051',   'New 0055',   'New 0068',   'New 0079',   'New 0100',   'New 0126',   'New 0138',   'New 0142',   'New 0153',   'New 0156',   'New 0162',   'New 0174',   'New 0178',   'New 0182'], 
'torse': ['New 0004',   'New 0050',   'New 0057',   'New 0069',   'New 0086',   'New 0097',   'New 0102',   'New 0110',   'New 0118',   'New 0119',   'New 0132',   'New 0145',   'New 0157',   'New 0183'], 
'G_epaule': ['New 0005', 'New 0039', 'New 0053', 'New 0148'], 
'D_epaule': ['New 0013', 'New 0014', 'New 0062', 'New 0084', 'New 0120'], 
'G_coude': ['New 0003',   'New 0019',   'New 0037',   'New 0054',   'New 0067',   'New 0127',   'New 0150'], 
'G_main': ['New 0010',   'New 0020',   'New 0041',   'New 0056',   'New 0081',   'New 0105',   'New 0116',   'New 0128',   'New 0149',   'New 0158', 'New 0165',  'New 0172'], 
'G_hanche': ['New 0002',   'New 0018',   'New 0025',   'New 0032',   'New 0040',   'New 0048',   'New 0052',   'New 0061',   'New 0064',   'New 0070',   'New 0075',   'New 0080',   'New 0083',   'New 0088',   'New 0093',   'New 0096',   'New 0101',   'New 0104',   'New 0107',   'New 0130',   'New 0136',   'New 0147',   'New 0154',   'New 0160',   'New 0166',   'New 0171',   'New 0179',   'New 0184'], 
'G_genou': ['New 0000',   'New 0033',   'New 0042',   'New 0046',   'New 0058',   'New 0066',   'New 0072',   'New 0129',   'New 0151',   'New 0155',   'New 0167',   'New 0185'], 
'G_pied': ['New 0011',   'New 0021',   'New 0043',   'New 0045',   'New 0059',   'New 0071',   'New 0090',   'New 0106',   'New 0114',  'New 0124', 'New 0131',   'New 0139',   'New 0152',   'New 0161',   'New 0168',   'New 0180',   'New 0186'], 
'D_genou': ['New 0006',   'New 0030',   'New 0034',   'New 0038',   'New 0044',   'New 0060',   'New 0076',   'New 0111',   'New 0123',   'New 0140',   'New 0163',   'New 0170',   'New 0175'], 
'D_pied': ['New 0008',   'New 0029',   'New 0065',   'New 0077',   'New 0089',   'New 0091',   'New 0094',   'New 0099',   'New 0108',   'New 0115',   'New 0134',   'New 0137',   'New 0146',   'New 0159',   'New 0177',   'New 0187'], 
'D_coude': ['New 0001', 'New 0015', 'New 0122', 'New 0143'], 
'D_main': ['New 0009',   'New 0017',   'New 0024',   'New 0027',   'New 0031',   'New 0036',   'New 0049',   'New 0085',   'New 0103',   'New 0109',   'New 0117',   'New 0125',   'New 0144',   'New 0164',   'New 0169',   'New 0173',   'New 0181'], 
'D_hanche': ['New 0012',   'New 0022',   'New 0026',   'New 0047',   'New 0063',   'New 0073',   'New 0074',   'New 0078',   'New 0082',   'New 0087',   'New 0092',   'New 0095',   'New 0112',   'New 0113',   'New 0121',   'New 0133',   'New 0135',   'New 0141',   'New 0176']}

connectl07={'tete': ['New 0005',   'New 0019',   'New 0021',   'New 0034',   'New 0044',   'New 0063',   'New 0079',   'New 0081',   'New 0106',   'New 0113',   'New 0127',   'New 0174',   'New 0186'],  
'torse': ['New 0000',   'New 0022',   'New 0026',   'New 0030',   'New 0038',   'New 0041',   'New 0049',   'New 0054',   'New 0056',   'New 0060',   'New 0068',   'New 0083',   'New 0092',   'New 0096',   'New 0109',   'New 0117',   'New 0128',   'New 0147',   'New 0166',   'New 0178',   'New 0191'],
'G_epaule': ['New 0009', 'New 0148', 'New 0149'], 
'D_epaule': ['New 0008', 'New 0051'], 
'G_coude': ['New 0013', 'New 0045', 'New 0065'], 
'G_main': ['New 0011',   'New 0040',   'New 0043',   'New 0047',   'New 0055',   'New 0064',   'New 0080',   'New 0094',   'New 0151',   'New 0167',   'New 0177',   'New 0183'], 
'G_hanche': ['New 0010',   'New 0015',   'New 0023',   'New 0027',   'New 0031',   'New 0033',   'New 0052',   'New 0057',   'New 0058',   'New 0067',   'New 0076',   'New 0086',   'New 0089',   'New 0098',   'New 0100',   'New 0110',   'New 0111',   'New 0122',   'New 0132',   'New 0133',   'New 0146',   'New 0159',   'New 0168',   'New 0173',   'New 0193'], 
'G_genou': ['New 0001',   'New 0017',   'New 0032',   'New 0050',   'New 0066',   'New 0071',   'New 0164',   'New 0184',   'New 0192'], 
'G_pied': ['New 0003',   'New 0016',   'New 0018',   'New 0028',   'New 0070',   'New 0077',   'New 0090',   'New 0101',   'New 0112',   'New 0134',   'New 0152',   'New 0182',   'New 0189'], 
'D_genou': ['New 0012',   'New 0037',   'New 0107',   'New 0116',   'New 0130',   'New 0139',   'New 0162',   'New 0175'], 
'D_pied': ['New 0014',   'New 0025',   'New 0062',   'New 0078',   'New 0091',   'New 0095',   'New 0102',   'New 0118',   'New 0140',   'New 0145',   'New 0160',   'New 0180',   'New 0181',   'New 0188'],  'D_hanche': ['New 0006',   'New 0039',   'New 0048',   'New 0061',   'New 0072',   'New 0075',   'New 0085',   'New 0088',   'New 0097',   'New 0099',   'New 0103',   'New 0119',   'New 0121',   'New 0131',   'New 0135',   'New 0144',   'New 0158',   'New 0161',   'New 0179',   'New 0187'], 
'D_coude': ['New 0002', 'New 0035', 'New 0114'], 
'D_main': ['New 0004',   'New 0020',   'New 0036',   'New 0042',   'New 0053',   'New 0084',   'New 0108',   'New 0115',   'New 0129',   'New 0137',   'New 0185'], 
'bassin': ['New 0007',   'New 0024',   'New 0029',   'New 0046',   'New 0059',   'New 0069',   'New 0073',   'New 0082',   'New 0087',   'New 0093',   'New 0120',   'New 0138',   'New 0157',   'New 0165',   'New 0170',   'New 0176',   'New 0190']} 

connectl08={'tete': ['New 0007', 'New 0023', 'New 0036', 'New 0046'], 
'torse': ['New 0001',   'New 0018',   'New 0025',   'New 0029',   'New 0037',   'New 0039',   'New 0040',   'New 0045',   'New 0048',   'New 0053',   'New 0060',   'New 0064',   'New 0073',   'New 0082',   'New 0095',   'New 0101',   'New 0109',   'New 0113',   'New 0117',   'New 0124',   'New 0128'], 
'G_epaule': ['New 0002', 'New 0020', 'New 0050'], 
'D_epaule': ['New 0004', 'New 0035'], 
'G_coude': ['New 0008', 'New 0021', 'New 0031', 'New 0093', 'New 0104'], 
'G_main': ['New 0010',   'New 0024',   'New 0033',   'New 0038',   'New 0044',   'New 0069',   'New 0091',   'New 0102'], 
'G_hanche': ['New 0013',   'New 0049',   'New 0052',   'New 0094',   'New 0105',   'New 0108',   'New 0112',   'New 0122',   'New 0125',   'New 0129'], 
'G_genou': ['New 0003', 'New 0026', 'New 0032'], 
'G_pied': ['New 0006',   'New 0028',   'New 0059',   'New 0063',   'New 0072',   'New 0087',   'New 0096',   'New 0106',   'New 0111',   'New 0116',   'New 0118'], 
'D_genou': ['New 0005', 'New 0067'], 
'D_pied': ['New 0009',   'New 0017',   'New 0027',   'New 0030',   'New 0043',   'New 0047',   'New 0054',   'New 0058',   'New 0061',   'New 0068',   'New 0076',   'New 0083',   'New 0088',   'New 0090',   'New 0100',   'New 0115',   'New 0119',   'New 0126',   'New 0130',   'New 0131'], 
'D_hanche': ['New 0011',   'New 0016',   'New 0019',   'New 0034',   'New 0051',   'New 0055',   'New 0057',   'New 0065',   'New 0075',   'New 0089',   'New 0098',   'New 0099',   'New 0107',   'New 0121',   'New 0123'], 
'D_coude': ['New 0000', 'New 0022'], 
'D_main': ['New 0014',   'New 0066',   'New 0070',   'New 0074',   'New 0081',   'New 0097',   'New 0120'], 
'bassin': ['New 0012',   'New 0015',   'New 0056',   'New 0092',   'New 0103',   'New 0110',   'New 0114',   'New 0127']} 

connectl09={'tete': ['New 0009', 'New 0024', 'New 0039', 'New 0062', 'New 0086'], 
'torse': ['New 0008',   'New 0020',   'New 0030',   'New 0033',   'New 0037',   'New 0048',   'New 0050',   'New 0055',   'New 0057',   'New 0061',   'New 0074',   'New 0081',   'New 0087',   'New 0097',   'New 0108',   'New 0112',   'New 0119',   'New 0137',   'New 0160',   'New 0171',   'New 0174',   'New 0181'], 
'G_epaule': ['New 0001', 'New 0025', 'New 0138'], 
'D_epaule': ['New 0003',   'New 0035',   'New 0043',   'New 0106',   'New 0140',   'New 0165',   'New 0178'], 
'G_coude': ['New 0013', 'New 0027', 'New 0083', 'New 0134', 'New 0150'], 
'G_main': ['New 0012',   'New 0028',   'New 0045',   'New 0053',   'New 0058',   'New 0073',   'New 0084',   'New 0095',   'New 0126',   'New 0142',   'New 0151'], 
'G_hanche': ['New 0011',   'New 0016',   'New 0026',   'New 0036',   'New 0047',   'New 0052',   'New 0059',   'New 0067',   'New 0072',   'New 0075',   'New 0090',   'New 0098',   'New 0100',   'New 0110',   'New 0123',   'New 0130',   'New 0135',   'New 0148',   'New 0163',   'New 0170',   'New 0175'],  
'G_genou': ['New 0004',   'New 0070',   'New 0077',   'New 0079',   'New 0136',   'New 0141',   'New 0152'], 
'G_pied': ['New 0006',   'New 0017',   'New 0023',   'New 0034',   'New 0046',   'New 0063',   'New 0069',   'New 0088',   'New 0091',   'New 0102',   'New 0143',   'New 0153',   'New 0159',   'New 0169',   'New 0173',   'New 0188'], 
'D_genou': ['New 0000',   'New 0041',   'New 0076',   'New 0080',   'New 0089',   'New 0093',   'New 0101',   'New 0115',   'New 0122',   'New 0131',   'New 0158',   'New 0166',   'New 0179'], 
'D_pied': ['New 0010',   'New 0022',   'New 0032',   'New 0044',   'New 0049',   'New 0066',   'New 0071',   'New 0078',   'New 0082',   'New 0092',   'New 0094',   'New 0096',   'New 0104',   'New 0109',   'New 0116',   'New 0124',   'New 0133',   'New 0145',   'New 0157',   'New 0162',   'New 0168',   'New 0172',   'New 0183',   'New 0184',   'New 0187'], 
'D_hanche': ['New 0002',   'New 0019',   'New 0038',   'New 0064',   'New 0085',   'New 0099',   'New 0111',   'New 0114',   'New 0121',   'New 0127',   'New 0129',   'New 0144',   'New 0146',   'New 0149',   'New 0155',   'New 0176',   'New 0182',   'New 0185'], 
'D_coude': ['New 0007', 'New 0042', 'New 0105', 'New 0167', 'New 0180'], 
'D_main': ['New 0014',   'New 0015',   'New 0018',   'New 0021',   'New 0029',   'New 0040',   'New 0051',   'New 0056',   'New 0060',   'New 0107',   'New 0113',   'New 0120',   'New 0125',   'New 0128',   'New 0139',   'New 0154',   'New 0164',   'New 0177'], 
'bassin': ['New 0005',   'New 0031',   'New 0054',   'New 0065',   'New 0068',   'New 0132',   'New 0147',   'New 0156',   'New 0161',   'New 0186']} 

connectl10={'tete': ['New 0010',   'New 0016',   'New 0022',   'New 0040',   'New 0053',   'New 0056',   'New 0067',   'New 0134'], 
'torse': ['New 0014',   'New 0023',   'New 0027',   'New 0031',   'New 0037',   'New 0041',   'New 0043',   'New 0054',   'New 0057',   'New 0068',   'New 0071',   'New 0080',   'New 0084',   'New 0100',   'New 0112',   'New 0121',   'New 0128',   'New 0139'], 
'G_epaule': ['New 0005'], 
'D_epaule': ['New 0003'], 
'G_coude': ['New 0013'], 
'G_main': ['New 0004', 'New 0070', 'New 0077', 'New 0127'], 
'G_hanche': ['New 0007',   'New 0032',   'New 0038',   'New 0045',   'New 0047',   'New 0051',   'New 0113',   'New 0116',   'New 0135'], 
'G_genou': ['New 0011',   'New 0015',   'New 0021',   'New 0065',   'New 0107',   'New 0114',   'New 0119',   'New 0137'], 
'G_pied': ['New 0000',   'New 0048',   'New 0052',   'New 0064',   'New 0075',   'New 0088',   'New 0096',   'New 0108',   'New 0115',   'New 0129',   'New 0136'], 
'D_genou': ['New 0008',   'New 0020',   'New 0026',   'New 0030',   'New 0033',   'New 0062',   'New 0079',   'New 0087',   'New 0094',   'New 0102',   'New 0124'], 
'D_pied': ['New 0002',   'New 0019',   'New 0028',   'New 0036',   'New 0059',   'New 0069',   'New 0076',   'New 0089',   'New 0095',   'New 0106',   'New 0122',   'New 0126',   'New 0132'], 
'D_hanche': ['New 0009',   'New 0034',   'New 0049',   'New 0058',   'New 0063',   'New 0073',   'New 0081',   'New 0086',   'New 0091',   'New 0099',   'New 0118',   'New 0130',   'New 0140'], 
'D_coude': ['New 0012', 'New 0024', 'New 0066'], 
'D_main': ['New 0006',   'New 0018',   'New 0025',   'New 0035',   'New 0039',   'New 0082',   'New 0085',   'New 0093',   'New 0101',   'New 0111',   'New 0125',   'New 0133'], 
'bassin': ['New 0001',   'New 0017',   'New 0029',   'New 0042',   'New 0044',   'New 0046',   'New 0050',   'New 0055',   'New 0060',   'New 0061',   'New 0072',   'New 0074',   'New 0078',   'New 0090',   'New 0103',   'New 0117',   'New 0120',   'New 0123',   'New 0131',   'New 0138']}

connectl11={'tete': ['New 0009',   'New 0027',   'New 0035',   'New 0052',   'New 0151',   'New 0171'],  
'torse': ['New 0012',   'New 0030',   'New 0036',   'New 0048',   'New 0059',   'New 0067',   'New 0072',   'New 0081',   'New 0089',   'New 0106',   'New 0116',   'New 0126',   'New 0140',   'New 0153',   'New 0169',   'New 0174',   'New 0187',   'New 0193',   'New 0202',   'New 0214'], 
'G_epaule': ['New 0013',   'New 0034',   'New 0053',   'New 0061',   'New 0172',   'New 0185'], 
'D_epaule': ['New 0000', 'New 0051', 'New 0136', 'New 0149', 'New 0208'], 
'G_coude': ['New 0006', 'New 0040', 'New 0055', 'New 0062', 'New 0186'], 
'G_main': ['New 0001',   'New 0023',   'New 0039',   'New 0056',   'New 0064',   'New 0070',   'New 0073',   'New 0076',   'New 0090',   'New 0104',   'New 0160'], 
'G_hanche': ['New 0002',   'New 0016',   'New 0022',   'New 0026',   'New 0041',   'New 0047',   'New 0057',   'New 0063',   'New 0069',   'New 0078',   'New 0085',   'New 0100',   'New 0114',   'New 0117',   'New 0124',   'New 0142',   'New 0144',   'New 0158',   'New 0159',   'New 0161',   'New 0176',   'New 0190',   'New 0194',   'New 0199',   'New 0211',   'New 0213'], 
'G_genou': ['New 0004',   'New 0024',   'New 0042',   'New 0050',   'New 0054',   'New 0084',   'New 0095',   'New 0097',   'New 0108',   'New 0113',   'New 0192',   'New 0198',   'New 0215'], 
'G_pied': ['New 0003',   'New 0019',   'New 0043',   'New 0058',   'New 0080',   'New 0092',   'New 0103',   'New 0109',   'New 0129',   'New 0139',   'New 0148',   'New 0155',   'New 0168',   'New 0173',   'New 0189',   'New 0200',   'New 0205',   'New 0212',   'New 0217'], 
'D_genou': ['New 0014',   'New 0018',   'New 0028',   'New 0122',   'New 0133',   'New 0141',   'New 0156',   'New 0201',   'New 0206'], 
'D_pied': ['New 0008',   'New 0021',   'New 0032',   'New 0044',   'New 0083',   'New 0086',   'New 0093',   'New 0098',   'New 0107',   'New 0115',   'New 0128',   'New 0130',   'New 0138',   'New 0154',   'New 0164',   'New 0167',   'New 0178',   'New 0184',   'New 0191',   'New 0197',   'New 0210'], 
'D_hanche': ['New 0015',   'New 0020',   'New 0029',   'New 0033',   'New 0045',   'New 0066',   'New 0088',   'New 0091',   'New 0101',   'New 0102',   'New 0105',   'New 0112',   'New 0119',   'New 0120',   'New 0127',   'New 0132',   'New 0143',   'New 0145',   'New 0157',   'New 0162',   'New 0175',   'New 0182',   'New 0188',   'New 0196',   'New 0204'], 
'D_coude': ['New 0010',   'New 0025',   'New 0137',   'New 0150',   'New 0170',   'New 0181',   'New 0209'], 
'D_main': ['New 0005',   'New 0017',   'New 0031',   'New 0038',   'New 0071',   'New 0087',   'New 0099',   'New 0118',   'New 0121',   'New 0125',   'New 0131',   'New 0135',   'New 0152',   'New 0207'],  
'bassin': ['New 0007',   'New 0037',   'New 0046',   'New 0049',   'New 0065',   'New 0077',   'New 0082',   'New 0094',   'New 0096',   'New 0110',   'New 0111',   'New 0163',   'New 0166',   'New 0179',   'New 0183',   'New 0195',   'New 0203',   'New 0216']} 

connectl12={'tete': ['New 0010',   'New 0026',   'New 0034',   'New 0038',   'New 0041',   'New 0045',   'New 0052',   'New 0054',   'New 0058',   'New 0063',   'New 0076',   'New 0078',   'New 0084',   'New 0091',   'New 0100',   'New 0108',   'New 0118',   'New 0128',   'New 0140',   'New 0153',   'New 0168',   'New 0169'], 
'torse': ['New 0006',   'New 0016',   'New 0025',   'New 0030',   'New 0033',   'New 0036',   'New 0040',   'New 0046',   'New 0051',   'New 0055',   'New 0062',   'New 0070',   'New 0075',   'New 0077',   'New 0127',   'New 0141',   'New 0147',   'New 0161',   'New 0166'], 
'G_epaule': ['New 0011'], 
'D_epaule': ['New 0002', 'New 0124'], 
'G_coude': ['New 0000', 'New 0044'], 
'G_main': ['New 0003', 'New 0017', 'New 0043', 'New 0047'], 
'G_hanche': ['New 0012',   'New 0028',   'New 0032',   'New 0035',   'New 0037',   'New 0050',   'New 0065',   'New 0068',   'New 0069',   'New 0123',   'New 0133',   'New 0136',   'New 0162'], 
'G_genou': ['New 0005',   'New 0018',   'New 0021',   'New 0029',   'New 0042',   'New 0053',   'New 0092',   'New 0125',   'New 0159',   'New 0165'], 
'G_pied': ['New 0009',   'New 0019',   'New 0023',   'New 0105',   'New 0115',   'New 0135',   'New 0145',   'New 0155',   'New 0163'], 
'D_genou': ['New 0015', 'New 0024', 'New 0088', 'New 0097', 'New 0149'], 
'D_pied': ['New 0008',   'New 0059',   'New 0064',   'New 0072',   'New 0079',   'New 0083',   'New 0090',   'New 0101',   'New 0109',   'New 0122',   'New 0132',   'New 0138',   'New 0143',   'New 0146',   'New 0150',   'New 0152',   'New 0156'], 
'D_coude': ['New 0007', 'New 0022'], 
'D_main': ['New 0014',   'New 0049',   'New 0087',   'New 0098',   'New 0107',   'New 0158',   'New 0164'], 
'bassin': ['New 0001',   'New 0027',   'New 0048',   'New 0056',   'New 0066',   'New 0074',   'New 0081',   'New 0116',   'New 0134',   'New 0142',   'New 0154',   'New 0160',   'New 0167',   'New 0170',   'New 0171'],
'D_hanche': ['New 0004',   'New 0020',   'New 0031',   'New 0039',   'New 0057',   'New 0060',   'New 0061',   'New 0067',   'New 0071',   'New 0073',   'New 0080',   'New 0082',   'New 0089',   'New 0093',   'New 0099',   'New 0106',   'New 0110',   'New 0111',   'New 0121',   'New 0126',   'New 0137',   'New 0139',   'New 0148',   'New 0151',   'New 0157']} 

def get_first_from_value(dico, val):
    return [v[0] for k, v in dico.items() if val in v]

def verif(connect, connectl, n_frames):
    success=0
    error=0
    for i in connect:
        for j in connect[i]:
            #point=1
            point=len(labdico[j])
            if get_first_from_value(connect, j) == get_first_from_value(connectl, j):
                success+=point
    return success/(15*n_frames)

In [ ]:
#Voici l'algo pour trouver connectl à partir du labdicol d'un fichier labelisé à la main. Le dictionnaire de sortie n'est pas parfait, mais au moins 95% des connexions sont corrects

connectl={}
for i in labdicol:
    df=labdicol[i]
    a=df.index[0]
    liste=[]
    if len(df)!=n_framesl:
        for h in range(len(df.index)):
            if df.index[h]+1 != df.index[h+1]:
                for j in labdico:
                    if a==labdico[j].index[0] and df.index[h]==labdico[j].index[-1] and df.loc[a].x==labdico[j].iloc[0].x:
                    #if a==labdico[j].index[0] and df.loc[a].x==labdico[j].iloc[0].x: #detect certains trucs indetecable par la condition au dessus
                        #print(a, df.index[h], j, labdico[j].index[0], labdico[j].index[-1])
                        liste.append(j)
                #i est df.index[h+1] identique ?
                a=df.index[h+1]
            if df.index[h] == df.index[-1]-1:#a la fin de la df, on arrete
                break;
        for j in labdico:
            if a == labdico[j].index[0]:
                #print(j)
                liste.append(j)
    connectl[i] = liste

In [22]:
#Les fonctions suivantes calculent la vitesse et l'accélération, vous pouvez les exploiter pour avoir des infos supplémentaires pour faire les connexions

def avg_speed_before(traj):
    cpt=0
    speed=0
    step=1
    time =100
    for frame in range(labdico[traj].index[-time], labdico[traj].index[-1]-step, step):
        pos_future=labdico[traj].loc[frame+step]
        speed+=math.dist(labdico[traj].loc[frame], pos_future)
        cpt+=1
    return speed/(cpt*step)

def avg_speed_after(traj):
    cpt=0
    speed=0
    step=1
    time =100
    for frame in range(labdico[traj].index[0], labdico[traj].index[time]-step, step):
        pos_future=labdico[traj].loc[frame+step]
        speed+=math.dist(labdico[traj].loc[frame], pos_future)
        cpt+=1
    return speed/(cpt*step)

def avg_speed_before(traj):
    time =100
    frame_beg = labdico[traj].index[-time]
    frame_end = labdico[traj].index[-1]
    speed = math.dist(labdico[traj].loc[frame_beg], labdico[traj].loc[frame_end])
    return speed/time

def avg_speed_after(traj):
    time =100
    frame_beg = labdico[traj].index[0]
    frame_end = labdico[traj].index[time]
    speed = math.dist(labdico[traj].loc[frame_beg], labdico[traj].loc[frame_end])
    return speed/time

def speed_frame(traj, frame):
    return math.dist(labdico[traj].loc[frame], labdico[traj].loc[frame+1])

def avg_acc_before(traj):
    time =100
    speed_beg = speed_frame(traj, labdico[traj].index[-time])
    speed_end = speed_frame(traj, labdico[traj].index[-2])
    return (speed_end - speed_beg)/time

def avg_acc_after(traj):
    time =100
    speed_beg = speed_frame(traj, labdico[traj].index[0])
    speed_end = speed_frame(traj, labdico[traj].index[time])
    return (speed_end - speed_beg)/time

In [9]:
for metric in ['d_space*d_time', '(d_space**2)*d_time', '(d_space**3)*d_time', '(d_space**4)*d_time', '(d_space**5)*d_time', '(d_space**6)*d_time', '(d_space**7)*d_time', '(d_space**8)*d_time', '(d_space**9)*d_time']:

    score_final=0

    for h in ['03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
        file='Measurement'+h+'.c3d'
        n_frames, lab, labdico = getData('Measurements_unlab\\'+file)
        #n_framesl, labl, labdicol = getData('Measurements_label\\'+file)

        artifacts=delete_artifacts(lab, labdico)
        lab=list(labdico.keys())
        
        '''
        for i in lab:
            if len(labdico[i])<n_frames/50:
                lab=[x for x in lab if x not in set(i)]
                del labdico[i]
                print(i)
        print(lab)
        '''

        #on créer la dataframe dist qui stocke les distances spatio-temporel entre marqueurs
        str1= "'" + "', '".join(lab) + "'"
        str2= "'" + "', '".join(lab) + "'"

        dist = pd.DataFrame()
        exec("dist = pd.DataFrame(index=["+ str1 +"], columns=["+ str2 +"])")

        for i in lab:
            for j in lab:
                if not superpos(labdico[i], labdico[j]):
                    if labdico[i].index[0] < labdico[j].index[0]:
                        d_space = math.dist(labdico[i].iloc[-1], labdico[j].iloc[0])
                        d_time = labdico[j].index[0]-labdico[i].index[-1]
                        exec("dist[j][i] = "+metric)

        dist=dist.apply(pd.to_numeric)
        

        connect ={}

        for i in lab:
            if 0==labdico[i].index[0] and n_frames==labdico[i].index[-1]+1:
                connect[i]=[i]
                dist=dist.drop(i)
                dist=dist.drop(i, axis=1)

        dist=dist.dropna(how='all').dropna(how='all', axis=1)

        #==========================================        
        d_max=dist.max(1).max()+1
        while (dist.min(1).min()<d_max):
            min=dist.stack().idxmin()
            connexion(min[0], min[1])
            dist=dist.drop(min[0])
            dist=dist.drop(min[1], axis=1)
        #==========================================
            
        exec("connectl = "+'connectl'+h+'.copy()')
        for k,val in connectl.items():
            connectl[k]=[e[5:] for e in val]

        score=verif(connect, connectl, n_frames)
        
        if h=='06':
            score=score*15/14
        
        print(file[:-4], score)
        score_final+=score

    print('Total', score_final/10, '\n')

(d_space**3)*d_time*(d_acc**0.05)
Measurement03 0.6313878731387873
Measurement04 0.5438251086678599
Measurement05 0.7223193851409052
Measurement06 0.47723063863286613
Measurement07 0.5539966689829368
Measurement08 0.6504381608282013
Measurement09 0.5388336390846771
Measurement10 0.7587971279617208
Measurement11 0.2113889477142283
Measurement12 0.47445739006464616
Total 0.556267494021683 

(d_space**3)*d_time*(d_acc**0.1)
Measurement03 0.6313878731387873
Measurement04 0.5438251086678599
Measurement05 0.7255792769712497
Measurement06 0.5281424801234916
Measurement07 0.5539966689829368
Measurement08 0.6504381608282013
Measurement09 0.5493547037430402
Measurement10 0.7321434261197268
Measurement11 0.30531545767019075
Measurement12 0.47445739006464616
Total 0.5694640546310131 

(d_space**3)*d_time*(d_acc**0.15)
Measurement03 0.6313878731387873
Measurement04 0.5438251086678599
Measurement05 0.7255792769712497
Measurement06 0.5281424801234916
Measurement07 0.5539966689829368
Measurement08 0.6